# Produccion electrica en base a la metereología

La idea principal del trabajo es ser capaces de predecir como va a afectar a que aumenten la produccion electrica de energias renovables las precipitaciones, el viento y las horas de sol.
Como las horas de sol y el viento son fenomenos cuya causa efecto es prácticamente inmediato vamos a centrarnos en la produccion de energia hidroelectrica.

En base a esto tenemos varias cuestiones que queremos despejar:
- Cuanto tarda en notarse en la generación eléctrica de fuentes de energía renovables los fenomenos metereológicos.
- Que cantidad de precipitaciones son necesarias para aumentar la generación de energia eléctica.
- Existe realmente una relación entre la metereologia y la producción electrica.


Como premisas partimos de :
- Vamos a considerar solo el poll de energia que proporciona Red Electrica de España (REE)
- Vamos a considerar que las empresas no trabajan bajo mala praxis y que intentan optimizar el uso de energias renovables.
- Se va a dividir el estudio por provincias, comunidades autonomas y a nivel estatal.

Como origenes de datos para el estudio vamos a utilizar los datos proporcionados por:
- Información de REE (https://www.ree.es/es/apidatos) obtenida mediante su API.
- Datos proporcionados por aemet(https://opendata.aemet.es/centrodedescargas/inicio), vamos a utilizar la libreria  aemet desarrollada por Pablo Moreno (https://pypi.org/project/python-aemet/).

Como requisitos para la ejecución del proyecto es necesario la instalación de la libreria python Aemet(pip install python-aemet) e instalar la libreria request



In [1]:
from aemet import Aemet,Estacion
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import json
import requests
import time

# Leemos la clave de la API de AEMET desde un fichero ubicado en la misma ruta que este notebook
with open('API_KEY_AEMET','r') as file:
    API_KEY_AE=file.read()

    

In [2]:
# Obtenemos la lista de estaciones de medicion de aemet 
info_estaciones=Estacion.get_estaciones(API_KEY_AE)

# Creamos un objeto Aemet para usar los metodos de la libreria aemet
aemet=Aemet(API_KEY_AE)


In [3]:
# Lectura de datos de AEMET


# Definimos funciones que vamos a utilizar para leer los datos de aement
def estaciones_prov (prov,lista_estaciones):
    '''Dada una provincia y una lista de diccionarios de estaciones de aemet. 
    Obtiene una lista de los ID de las estaciones de esa provincia.'''
    lista_id=[]
    for estacion in lista_estaciones:
        if estacion['provincia']==prov.upper():
            lista_id.append(estacion['indicativo'])
    return lista_id

help(estaciones_prov)


def lectura_dias(date_ini,date_end,estaciones):
    '''Dado un json del objeto Estacion, y fechas de inicio y fin:
    Obtenemos los datos climatologicos entre las dos fechas para todas las estaciones de manera diaria
    Si la fecha de inicio es anterior a 2016, se cambia a 2016-01-01, para evitar errores.
    '''
    valores_diarios=[]
    if date_ini[0:4]<'2016-01':
        date_ini="2016-01-01T00:00:00UTC"
    if date_ini>date_end:
        print('Valores no válidos, fecha de inicio mayor que la fecha de fin')
        return valores_diarios;
    for element in estaciones:
        try:
            valores_estacion=aemet.get_valores_climatologicos_diarios(date_ini,date_end,element['indicativo'])
            if type(valores_estacion)!=dict:
                valores_diarios.extend(valores_estacion)
        except:
            time.sleep(0.1) # para evitar errores por nº de lecturas.
    return valores_diarios;

help(lectura_dias)

# Pasamos los datos ha dataframes para su procesado
estaciones=pd.DataFrame(info_estaciones)

date_ini="2020-12-30T00:00:00UTC"
date_end="2020-12-31T00:00:00UTC"

wheather=pd.DataFrame(lectura_dias(date_ini,date_end,info_estaciones))

print(max(wheather['fecha']),min(wheather['fecha']))


#Rellenamos los NaN por 0 para precipitaciones. 
#Y para temperatura los NaN por la media entre la termperatura del dia anterior y la temperatura del dia siguiente


Help on function estaciones_prov in module __main__:

estaciones_prov(prov, lista_estaciones)
    Dada una provincia y una lista de diccionarios de estaciones de aemet. 
    Obtiene una lista de los ID de las estaciones de esa provincia.

Help on function lectura_dias in module __main__:

lectura_dias(date_ini, date_end, estaciones)
    Dado un json del objeto Estacion, y fechas de inicio y fin:
    Obtenemos los datos climatologicos entre las dos fechas para todas las estaciones de manera diaria
    Si la fecha de inicio es anterior a 2016, se cambia a 2016-01-01, para evitar errores.

2020-12-31 2020-12-30


In [60]:
wheather

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin
0,2020-12-30,0252D,ARENYS DE MAR,BARCELONA,74,"8,6","0,0","3,7",06:30,"13,4",12:30,24,"1,1","6,7",11:10,NaN,NaN,NaN,NaN,NaN
1,2020-12-31,0252D,ARENYS DE MAR,BARCELONA,74,"9,2","0,0","4,8",04:00,"13,5",13:00,24,"2,8","9,4",13:40,NaN,NaN,NaN,NaN,NaN
2,2020-12-30,0076,BARCELONA AEROPUERTO,BARCELONA,4,"9,2","0,0","5,1",07:17,"13,3",12:40,32,"6,1","13,3",03:15,"5,4","1014,1",Varias,"1004,2",01
3,2020-12-31,0076,BARCELONA AEROPUERTO,BARCELONA,4,"8,7","0,0","3,6",06:07,"13,8",12:04,24,"4,2","10,8",13:22,"5,1","1014,1",01,"1006,1",24
4,2020-12-30,0200E,"BARCELONA, FABRA",BARCELONA,408,"5,8","0,0","2,3",Varias,"9,4",12:46,33,"6,7","15,0",04:07,"6,9","964,1",24,"954,9",Varias
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,2020-12-31,B278,"PALMA DE MALLORCA, AEROPUERTO",ILLES BALEARS,8,"8,0","0,0","1,9",07:16,"14,0",11:07,25,"3,9","13,3",23:47,"8,2","1015,2",08,"1007,8",24
83,2020-12-30,B228,"PALMA, PUERTO",ILLES BALEARS,3,"11,2","0,0","8,9",06:40,"13,5",14:00,35,"2,2","14,4",11:00,"6,8","1015,1",22,"1005,8",00
84,2020-12-31,B228,"PALMA, PUERTO",ILLES BALEARS,3,"11,6","1,0","7,9",21:20,"15,2",11:40,28,"1,1","8,6",23:40,"4,4","1015,1",08,"1007,5",24
85,2020-12-30,B346X,PORRERES,ILLES BALEARS,120,"9,1","0,0","5,7",00:40,"12,5",13:00,31,"3,1","13,1",09:50,NaN,NaN,NaN,NaN,NaN


In [4]:
# Leemos las regiones de ree obtenidas desde (https://www.ree.es/es/apidatos) desde un fichero ubicado en la misma ruta que este notebook
region_ree=pd.read_csv('REGION_REE',header=0,index_col='Region')
region_ccaa=region_ree[region_ree.geo_limit=='ccaa']

region_ccaa.iloc[1]

geo_limit    ccaa
geo_id          5
Name: Aragón, dtype: object

In [5]:
url_prueba='https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date=2014-01-01T00:00&end_date=2018-12-31T23:59&time_trunc=year&geo_trunc=electric_system&geo_limit=ccaa&geo_ids=7'
ree_bal=requests.get(url_prueba)
ree_bal.json()

{'data': {'type': 'Generación por tecnología',
  'id': 'gen1',
  'attributes': {'title': 'Generación por tecnología',
   'last-update': None,
   'description': None},
  'meta': {'cache-control': {'cache': 'MISS'}}},
 'included': [{'type': 'Hidráulica',
   'id': '10330',
   'groupId': '1',
   'attributes': {'title': 'Hidráulica',
    'description': None,
    'color': '#0090d1',
    'type': 'Renovable',
    'magnitude': None,
    'composite': False,
    'last-update': '2019-11-29T14:14:06.000+01:00',
    'values': [{'value': 1006208.552,
      'percentage': 0.044706612510726725,
      'datetime': '2014-01-01T00:00:00.000+01:00'},
     {'value': 700914.375,
      'percentage': 0.03246965233571151,
      'datetime': '2015-01-01T00:00:00.000+01:00'},
     {'value': 731944.331,
      'percentage': 0.034389006748994,
      'datetime': '2016-01-01T00:00:00.000+01:00'},
     {'value': 410077.844,
      'percentage': 0.019136375980118164,
      'datetime': '2017-01-01T00:00:00.000+01:00'},
     

### Lectura de los datos de REE

Para la lectura de los datos de REE voy a utilizar la libreria python requests para a traves de su API, obtener la generación de electricidad en , para cada tipo de energia Electrica.

Una vez leidos los datos de la API, guardo los datos en formato json en un fichero. para no tener que repetir las consultas y poder trabajar sin conexion.

La estrutura de los datos leidos de REE es la siguiente:


Por lo que vamos a almacenar 2 ficheros:
    - Renovables
    - No Renovables



In [138]:
# Obtenemos los datos de REE a traves de su API.
https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date=2018-01-01T00:00&end_date=2018-12-31T23:59&time_trunc=month&geo_trunc=electric_system&geo_limit=ccaa&geo_ids=7
# meter esto en una funcion con su try-exception
parametros={'start_date':'2020-12-31T00:00','end_date':'2021-01-01T10:00','time_trunc':'day','geo_limit':'ccaa','geo_id':'6'}
URL_BAL='https://apidatos.ree.es/es/datos/generacion/estructura-generacion'

ree_bal=requests.get(URL_BAL,params=parametros)
  
if ree_bal.raise_for_status()!=None:
    print('Error en lectura')


In [139]:
print(ree_bal.url)
print(ree_bal.status_code)
print(ree_bal.elapsed)
print(ree_bal.encoding)
print(ree_bal.status_code)
print(ree_bal.ok)
print(ree_bal.history)

https://apidatos.ree.es/es/datos/balance/balance-electrico?start_date=2021-01-01T00%3A00&end_date=2021-01-01T10%3A00&time_trunc=day&geo_limit=ccaa&geo_id=6
200
0:00:00.424561
None
200
True
[]


In [140]:
with open('ree_bal.json','w') as file:
    json.dump(ree_bal.json()['included'][0]['attributes']['content'],file)
    
42181.4

42181.4

In [141]:
ree_bal.json()['included']

[{'type': 'Renovable',
  'id': 'Renovable',
  'attributes': {'title': 'Renovable',
   'last-update': '2021-01-04T07:34:28.000+01:00',
   'description': None,
   'magnitude': None,
   'content': [{'type': 'Hidráulica',
     'id': '10288',
     'groupId': 'Renovable',
     'attributes': {'title': 'Hidráulica',
      'description': None,
      'color': '#0090d1',
      'type': 'distinct',
      'magnitude': None,
      'composite': False,
      'last-update': '2021-01-04T07:34:28.000+01:00',
      'values': [{'value': 42181.4,
        'percentage': 0.27126590438118353,
        'datetime': '2021-01-01T00:00:00.000+01:00'}],
      'total': 42181.4}},
    {'type': 'Eólica',
     'id': '10291',
     'groupId': 'Renovable',
     'attributes': {'title': 'Eólica',
      'description': None,
      'color': '#6fb114',
      'type': 'distinct',
      'magnitude': None,
      'composite': False,
      'last-update': '2021-01-04T07:34:29.000+01:00',
      'values': [{'value': 101886.352,
        'per

In [134]:
pd.read_json('ree_bal.json')

,type,id,groupId,attributes
0,Hidráulica,10288,Renovable,"{'title': 'Hidráulica', 'description': None, '..."
1,Eólica,10291,Renovable,"{'title': 'Eólica', 'description': None, 'colo..."
2,Solar fotovoltaica,1458,Renovable,"{'title': 'Solar fotovoltaica', 'description':..."
3,Solar térmica,1459,Renovable,"{'title': 'Solar térmica', 'description': None..."
4,Hidroeólica,1455,Renovable,"{'title': 'Hidroeólica', 'description': None, ..."
5,Otras renovables,10292,Renovable,"{'title': 'Otras renovables', 'description': '..."
6,Residuos renovables,10295,Renovable,"{'title': 'Residuos renovables', 'description'..."
7,Generación renovable,Renovable,Renovable,"{'title': 'Generación renovable', 'description..."


In [85]:
renovable=data ['included'][0]
no_renovable=data ['included'][1]
demanda=data ['included'][2]

In [91]:
renovable['attributes']



{'title': 'Renovable',
 'last-update': '2020-01-10T08:15:21.000+01:00',
 'description': None,
 'magnitude': None,
 'content': [{'type': 'Hidráulica',
   'id': '10288',
   'groupId': 'Renovable',
   'attributes': {'title': 'Hidráulica',
    'description': None,
    'color': '#0090d1',
    'type': 'distinct',
    'magnitude': None,
    'composite': False,
    'last-update': '2020-01-10T08:15:21.000+01:00',
    'values': [{'value': 19307.962,
      'percentage': 0.29485581120822324,
      'datetime': '2019-01-01T00:00:00.000+01:00'}],
    'total': 19307.962}},
  {'type': 'Eólica',
   'id': '10291',
   'groupId': 'Renovable',
   'attributes': {'title': 'Eólica',
    'description': None,
    'color': '#6fb114',
    'type': 'distinct',
    'magnitude': None,
    'composite': False,
    'last-update': '2020-01-10T08:15:42.000+01:00',
    'values': [{'value': 37895.955,
      'percentage': 0.57871682951496,
      'datetime': '2019-01-01T00:00:00.000+01:00'}],
    'total': 37895.955}},
  {'type

In [95]:
drenov=pd.DataFrame.from_dict(renovable['attributes'])
drenov

,title,last-update,description,magnitude,content
0,Renovable,2020-01-10T08:15:21.000+01:00,None,None,"{'type': 'Hidráulica', 'id': '10288', 'groupId..."
1,Renovable,2020-01-10T08:15:21.000+01:00,None,None,"{'type': 'Eólica', 'id': '10291', 'groupId': '..."
2,Renovable,2020-01-10T08:15:21.000+01:00,None,None,"{'type': 'Solar fotovoltaica', 'id': '1458', '..."
3,Renovable,2020-01-10T08:15:21.000+01:00,None,None,"{'type': 'Solar térmica', 'id': '1459', 'group..."
4,Renovable,2020-01-10T08:15:21.000+01:00,None,None,"{'type': 'Hidroeólica', 'id': '1455', 'groupId..."
5,Renovable,2020-01-10T08:15:21.000+01:00,None,None,"{'type': 'Otras renovables', 'id': '10292', 'g..."
6,Renovable,2020-01-10T08:15:21.000+01:00,None,None,"{'type': 'Residuos renovables', 'id': '10295',..."
7,Renovable,2020-01-10T08:15:21.000+01:00,None,None,"{'type': 'Generación renovable', 'id': 'Renova..."
